# BigQuery Exploration with bq-* Utilities

This notebook demonstrates how to use the BigQuery utilities for data exploration.

We'll explore public datasets using:
- `bq-profile` - Generate comprehensive data profiles
- `bq-lineage` - Explore table dependencies
- `bq-schema-diff` - Compare table schemas

## Setup

In [ ]:
import subprocess
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from pathlib import Path

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Initialize BigQuery client
client = bigquery.Client()

# Path to utilities
UTILS_DIR = Path('../bin/data-utils')

print('Setup complete!')

## Helper Functions

In [ ]:
def run_util(util_name, args, parse_json=True):
    """Run a bq-* utility and return output."""
    util_path = UTILS_DIR / util_name
    cmd = [str(util_path)] + args
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f'Error: {result.stderr}')
        return None
    
    if parse_json:
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError:
            print('Warning: Could not parse JSON output')
            return result.stdout
    return result.stdout

def display_profile_summary(profile):
    """Display key metrics from a table profile."""
    meta = profile['table_overview']
    
    print(f"Table: {profile['table_id']}")
    print(f"Rows: {meta['num_rows']:,}")
    print(f"Size: {meta['num_bytes']:,} bytes")
    print(f"Columns: {meta['num_columns']}")
    print(f"Created: {meta['created']}")
    print()

## 1. Profile Public Dataset: Stack Overflow

Let's profile the Stack Overflow posts table from BigQuery's public datasets.

In [ ]:
# Profile the Stack Overflow posts table
table_id = 'bigquery-public-data.stackoverflow.posts_questions'

print('Profiling Stack Overflow posts table...')
profile = run_util('bq-profile', [table_id, '--format=json', '--sample-size=5'])

if profile:
    display_profile_summary(profile)

### Visualize Data Type Distribution

In [ ]:
if profile:
    # Create DataFrame from data type distribution
    type_dist = profile['data_type_distribution']
    df_types = pd.DataFrame([
        {'Type': k, 'Count': v} for k, v in type_dist.items()
    ])
    
    # Plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_types, x='Type', y='Count', palette='viridis')
    plt.title('Data Type Distribution')
    plt.xlabel('Data Type')
    plt.ylabel('Number of Columns')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

### Analyze Column Statistics

In [ ]:
if profile:
    col_stats = profile['column_statistics']
    
    # Create DataFrame of null percentages
    null_data = []
    for col_name, stats in col_stats.items():
        null_data.append({
            'Column': col_name,
            'Null %': stats.get('null_percentage', 0),
            'Uniqueness': stats.get('uniqueness_ratio', 0)
        })
    
    df_nulls = pd.DataFrame(null_data).sort_values('Null %', ascending=False)
    
    # Plot null percentages
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_nulls, x='Column', y='Null %', palette='coolwarm')
    plt.title('Null Percentage by Column')
    plt.xlabel('Column')
    plt.ylabel('Null %')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()
    
    print('\nColumns with highest null percentages:')
    print(df_nulls.head(5))

### Visualize Uniqueness Ratios

In [ ]:
if profile:
    # Plot uniqueness ratios
    df_unique = df_nulls.sort_values('Uniqueness', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_unique, x='Column', y='Uniqueness', palette='magma')
    plt.title('Column Uniqueness Ratio (1.0 = all unique values)')
    plt.xlabel('Column')
    plt.ylabel('Uniqueness Ratio')
    plt.xticks(rotation=90)
    plt.axhline(y=1.0, color='r', linestyle='--', label='Perfect Uniqueness')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print('\nPotential primary key candidates (uniqueness >= 0.99):')
    pk_candidates = df_unique[df_unique['Uniqueness'] >= 0.99]
    print(pk_candidates[['Column', 'Uniqueness']])

## 2. Schema Comparison: Compare Two Versions

Let's compare schemas of different tables to understand schema evolution.

In [ ]:
# Compare two public dataset tables
table_a = 'bigquery-public-data.stackoverflow.posts_questions'
table_b = 'bigquery-public-data.stackoverflow.posts_answers'

print(f'Comparing schemas:\n  A: {table_a}\n  B: {table_b}\n')

schema_diff = run_util('bq-schema-diff', [table_a, table_b, '--format=json'])

if schema_diff:
    print(f"Identical: {schema_diff['identical']}")
    print(f"\nSummary:")
    print(f"  Fields only in A: {schema_diff['summary']['fields_only_in_a']}")
    print(f"  Fields only in B: {schema_diff['summary']['fields_only_in_b']}")
    print(f"  Type changes: {schema_diff['summary']['type_changes']}")

In [ ]:
if schema_diff and not schema_diff['identical']:
    # Display fields only in A
    if schema_diff['only_in_a']:
        print('\nFields only in Table A:')
        df_a = pd.DataFrame(schema_diff['only_in_a'])
        display(df_a)
    
    # Display fields only in B
    if schema_diff['only_in_b']:
        print('\nFields only in Table B:')
        df_b = pd.DataFrame(schema_diff['only_in_b'])
        display(df_b)
    
    # Display type changes
    if schema_diff['type_changes']:
        print('\nType Changes:')
        df_changes = pd.DataFrame(schema_diff['type_changes'])
        display(df_changes)

## 3. Lineage Analysis

Explore table dependencies to understand data lineage.

In [ ]:
# Note: Lineage analysis works best with views and materialized views
# For this demo, we'll use a hypothetical example structure

# Example: Analyze lineage of a table
# Uncomment and replace with your own table that has dependencies
# lineage_table = 'your-project.your-dataset.your-view'
# lineage = run_util('bq-lineage', [lineage_table, '--format=json'])

# if lineage:
#     print(f"Table: {lineage['table_id']}")
#     print(f"Direction: {lineage['direction']}")
#     print(f"\nUpstream dependencies: {len(lineage['upstream'])}")
#     for table in lineage['upstream']:
#         print(f"  <- {table}")
#     
#     print(f"\nDownstream dependencies: {len(lineage['downstream'])}")
#     for table in lineage['downstream']:
#         print(f"  -> {table}")

print('Lineage analysis example (replace with your own view/table):')
print('Uncomment the code above and provide a table with dependencies')

## 4. Multi-Table Profiling

Profile multiple tables and compare key metrics.

In [ ]:
# Profile multiple related tables
tables_to_profile = [
    'bigquery-public-data.stackoverflow.posts_questions',
    'bigquery-public-data.stackoverflow.posts_answers',
]

profiles = {}
for table in tables_to_profile:
    print(f'Profiling {table}...')
    profile = run_util('bq-profile', [table, '--format=json'])
    if profile:
        profiles[table] = profile
        display_profile_summary(profile)

### Compare Table Sizes and Row Counts

In [ ]:
if profiles:
    # Extract metrics for comparison
    comparison_data = []
    for table_id, profile in profiles.items():
        meta = profile['table_overview']
        # Shorten table names for display
        short_name = table_id.split('.')[-1]
        comparison_data.append({
            'Table': short_name,
            'Rows': meta['num_rows'],
            'Size (GB)': meta['num_bytes'] / (1024**3),
            'Columns': meta['num_columns']
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Row counts
    sns.barplot(data=df_comparison, x='Table', y='Rows', ax=axes[0], palette='Set2')
    axes[0].set_title('Row Count Comparison')
    axes[0].set_ylabel('Rows')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Table sizes
    sns.barplot(data=df_comparison, x='Table', y='Size (GB)', ax=axes[1], palette='Set2')
    axes[1].set_title('Table Size Comparison')
    axes[1].set_ylabel('Size (GB)')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Column counts
    sns.barplot(data=df_comparison, x='Table', y='Columns', ax=axes[2], palette='Set2')
    axes[2].set_title('Column Count Comparison')
    axes[2].set_ylabel('Columns')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print('\nTable Comparison:')
    display(df_comparison)

## 5. Query and Visualize Actual Data

Combine utility insights with actual data queries.

In [ ]:
# Query sample data from Stack Overflow
query = """
SELECT 
    EXTRACT(YEAR FROM creation_date) as year,
    COUNT(*) as question_count
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE creation_date >= '2015-01-01'
GROUP BY year
ORDER BY year
LIMIT 10
"""

df_questions = client.query(query).to_dataframe()

# Plot questions over time
plt.figure(figsize=(10, 6))
plt.plot(df_questions['year'], df_questions['question_count'], marker='o', linewidth=2)
plt.title('Stack Overflow Questions by Year')
plt.xlabel('Year')
plt.ylabel('Number of Questions')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

display(df_questions)

## Summary

This notebook demonstrated:

1. **Table Profiling** - Using `bq-profile` to analyze table structure and statistics
2. **Schema Comparison** - Using `bq-schema-diff` to identify schema differences
3. **Lineage Analysis** - Using `bq-lineage` to explore table dependencies
4. **Multi-Table Analysis** - Comparing metrics across multiple tables
5. **Data Visualization** - Combining utility outputs with data queries

### Next Steps

- Apply these techniques to your own BigQuery datasets
- Automate data quality checks using these utilities
- Create dashboards from the profiling data
- Build data catalogs using schema and lineage information